In [ ]:
import csv
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import os
import itertools
from IPython.display import display, HTML
import matplotlib.pyplot as plt

In [ ]:
def save_data_to_file(folder_name, file_name, data):
    try:
        # Create target Directory
        os.mkdir(folder_name)
        print("Directory " , folder_name ,  " Created ") 
    except FileExistsError:
        e = 1
        #print("Directory " , folder_name ,  " already exists")
    data.to_csv(folder_name + "/" + file_name, index=False)

In [ ]:
def save_data_to_file_with_index(folder_name, file_name, data):
    try:
        # Create target Directory
        os.mkdir(folder_name)
        print("Directory " , folder_name ,  " Created ") 
    except FileExistsError:
        e = 1
        #print("Directory " , folder_name ,  " already exists")
    data.to_csv(folder_name + "/" + file_name)

In [ ]:
features = [("Old_data/FrequencyOfUserPostsWithoutZeros","std_post_frequency"),
            ("Old_data/FrequencyOfUserPostsWithoutZeros","q3_post_frequency"),
            ("Old_data/NumberOfReceivedResponsesToUsersPostsWithoutZeros","number_of_received_responses_to_users_posts_std"),
            ("Old_data/NumberOfReceivedResponsesToUsersPostsWithoutZeros","number_of_received_responses_to_users_posts_max"),
            ("Old_data/NumberOfReceivedResponsesUnderUsersCommentsWithoutZeros", "number_of_received_responses_under_users_comments_q3"),
            ("Old_data/NumberOfReceivedResponsesUnderUsersCommentsWithoutZeros","number_of_received_responses_under_users_comments_max"),
            ("Old_data/NumberOfWordsInOwnResponsesOfUsersPostsWithoutZeros", "number_of_words_in_own_responses_of_users_posts_q3"),
            ("Old_data/NumberOfWordsInResponsesOfUsersPostsWithoutZeros","nnumber_of_words_in_responses_of_users_posts_median"),
            ("Old_data/SentimentOfUsersPostsWithoutZeros","posts_sentiment_min"),
            ("Old_data/FrequencyOfUserCommentsWithoutZeros", "mean_comments_frequency"),
            ("Old_data/NumberOfWordsInUsersCommentsWithoutZeros","number_of_words_in_users_comments_avg"),
            ("Old_data/NumberOfWordsInUsersPostsWithoutZeros", "number_of_words_in_users_posts_q3"),
            ("Old_data/NumberOfReceivedResponsesToUsersPostsWithoutZeros", "number_of_received_responses_to_users_posts_q3"),
            ("Old_data/NumberOfCommentsWrittenByUserUnderHisOwnPostsWithoutZeros", "number_of_comments_written_by_user_under_his_own_posts_q3")
           ]

In [ ]:
start_date_comments = datetime.date(2008, 12, 9)
end_date_comments = datetime.date(2013,11, 16)

In [ ]:
pd.read_csv("Old_data/NumberOfReceivedResponsesToUsersPostsWithoutZeros" + "/feature_" + str(start_date_comments) + ".csv")

In [ ]:
def merge_stats(start_date):
    statistics_to_join = []
    for feat in features:
        (file_name, feature) = feat
        statistics_to_join.append(pd.read_csv(file_name + "/feature_" + str(start_date) + ".csv")[['user_id',feature]])
    
    merged_df = statistics_to_join[0]
    for i in range(1, len(statistics_to_join)):
        merged_df = pd.merge(merged_df, statistics_to_join[i],how='outer',on=['user_id'])
    return merged_df.fillna(0)

In [ ]:
def create_cluster_data(start_date, end_date):
    res = merge_stats(start_date)
    start_date += timedelta(days=14)
    while start_date < end_date:
        data = merge_stats(start_date)
        res = pd.concat([res, data], ignore_index=True)
        print(str(start_date))
        #print(str(len(data)) + " " + str(start_date))
        
        start_date += timedelta(days=14)
    save_data_to_file("Cluster_Data_All", "cluster.csv", res)

In [ ]:
create_cluster_data(start_date_comments, end_date_comments)

#### Clustering with new names

In [ ]:
#mine k = 7

# columns = ['start_date',
#            'user_id',
#            'posts_activity_time',
#            'frequency_of_posts_avg',
#            'frequency_of_posts_stddev',
#            'frequency_of_comments_stddev',
#            'frequency_of_comments_q3',
#            'number_of_received_responses_to_post_max',
#            'number_of_received_responses_to_post_stddev',
#            'number_of_received_responses_to_post_avg',
#            'number_of_received_responses_under_comments_stddev',
#            'number_of_received_responses_under_comments_q3',
#            'number_of_received_responses_under_comments_max',
#            'number_of_own_post_responses_max', 
#            'number_of_words_in_posts_q3',
#            'number_of_words_in_comments_median',
#            'number_of_words_in_responses_to_posts_q3',
#            'number_of_words_in_own_post_responses_q3',
#            'number_of_all_responses_from_unique_users_in_slot']

# ,
#            'post_sentiment_stddev',
#            'post_sentiment_q3',
#            'comments_sentiment_q3',
#            'comments_sentiment_stddev',
#            'received_response_sentiment_stddev'


columns = ['start_date',
           'user_id',
           'posts_activity_time',
           'frequency_of_posts_avg',
           'frequency_of_posts_stddev',
           'frequency_of_comments_q3',
           'number_of_received_responses_to_post_max',
           'number_of_received_responses_to_post_stddev',
           'number_of_received_responses_to_post_avg',
           'number_of_received_responses_to_post_q3',
           'number_of_received_responses_under_comments_q3',
           'number_of_received_responses_under_comments_max',
           'number_of_own_post_responses_q3',
           'number_of_words_in_posts_q3',
           'number_of_words_in_comments_median',
           'number_of_words_in_responses_to_posts_q3',
           'number_of_words_in_own_post_responses_q3']

# #            'post_sentiment_median',
# #            'post_sentiment_min', 'comments_sentiment_median', 'received_response_sentiment_stddev'

In [ ]:
feature_table = pd.read_csv("FeatureTableCSV.csv", sep=";")

In [ ]:
start_date_comments = datetime.date(2009, 12, 22)
end_date_comments = datetime.date(2013,10, 22)

In [ ]:
def generate_stats_for_entire_data(feature_table, start_date, end_date):
    results = pd.DataFrame()
    while start_date < end_date:
        features = pd.read_csv("All_Data_In_Slots_Joined/" + str(start_date) + "_joined.csv")
        features.insert(1, 'start_date', start_date)
        results = pd.concat([results, features], ignore_index=True)
        start_date += timedelta(days=14)
    return results

In [ ]:
def save_clustering_data(columns, feature_table, start_date_comments, end_date_comments):
    alldata = generate_stats_for_entire_data(feature_table, start_date_comments, end_date_comments)[columns]
    save_data_to_file("Cluster_Data_All", "cluster.csv", alldata)

In [ ]:
save_clustering_data(columns, feature_table, start_date_comments, end_date_comments)

In [ ]:
start_date_clustering = datetime.date(2009, 12, 8)
end_date_clustering = datetime.date(2013,10, 22)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

In [ ]:
def benchmark_custering(X):
    columns=['k', 'score_mean', 'scored_std']
    results = pd.DataFrame(columns=columns)
    print("Benchmarking..")
    for k in range(4,11):
        scores = []
        for sample_num in range(0,10):
            kmeans = KMeans(n_clusters=k).fit(X)
            labels = kmeans.labels_
            scores.append(metrics.calinski_harabasz_score(X, labels))
            print("Clustering done for sample: " + str(sample_num))
        row = pd.DataFrame([[k, np.mean(scores), np.std(scores)]], columns=columns)
        results = pd.concat([row, results], ignore_index=True)
        print("Clustering done for: " + str(k))
    return results

In [ ]:
def generate_benchmark_clustering_data():
    df = pd.read_csv("Cluster_Data_All" + "/cluster.csv").drop(columns =["user_id", "start_date"])
    X = MinMaxScaler().fit_transform(df)
    results = benchmark_custering(X)
    save_data_to_file("Cluster_Benchmarks_All", "cluster_benchmark.csv", results)
    print("Results for {} saved")
    

In [ ]:
generate_benchmark_clustering_data()

In [ ]:

df = pd.read_csv("Cluster_Benchmarks_All/cluster_benchmark.csv")

x = df["k"]
y = df["score_mean"]
print(y)
print(df["scored_std"])
yerr = df["scored_std"]
plt.errorbar(x,y,yerr=yerr)
plt.show()
# plt.errorbar('k', 'calinski_harabaz_score', yerr='std', data=df, marker='o', linestyle='solid',
#              ecolor='darkred', capsize=3, markersize=3, capthick=2)
# plt.xticks(range(2,16))
# plt.xlabel('clusters number')
# plt.ylabel('Calinski Harabaz score')
# plt.title('K-means clustering evaluation')
# plt.show()
df

In [ ]:
def cluster_data():
    data_to_cluster = pd.read_csv("Cluster_Data_All" + "/cluster.csv")
    data_without_id = data_to_cluster.drop(columns =["user_id", "start_date"])
    
    X = MinMaxScaler().fit_transform(data_without_id)
    kmeans = KMeans(n_clusters=7).fit(data_without_id)
    labels_df = pd.DataFrame(kmeans.labels_, columns=["label"])
    labeled_users = pd.concat([data_to_cluster, labels_df], axis=1)
    cluster_df = pd.DataFrame(kmeans.cluster_centers_, columns=data_without_id.columns.values)
    
#     save_data_to_file("All_Labeled_users", "labeled_users.csv", labeled_users)
#     save_data_to_file("All_Cluster_centers", "cluster_centers.csv", cluster_df)

    save_data_to_file("All_Labeled_users", "with_new_feature_labeled_users.csv", labeled_users)
    save_data_to_file("All_Cluster_centers", "with_new_feature_cluster_centers.csv", cluster_df)
    
    

# for index, row in df.iterrows():
#     data_to_cluster = pd.read_csv("Cluster_Data" + "/cluster" + row["date"] + ".csv")
#     data_without_id = data_to_cluster.drop(columns =["user_id"])
#     data_to_cluster
#     X = MinMaxScaler().fit_transform(data_without_id)
#     kmeans = KMeans(n_clusters=7).fit(X)
#     labels_df = pd.DataFrame(kmeans.labels_, columns=["label"])
#     labeled_users = pd.concat([data_to_cluster, labels_df], axis=1)
#     cluster_df = pd.DataFrame(kmeans.cluster_centers_, columns=data_without_id.columns.values)
    
#     save_data_to_file("Labeled_users", "labeled_users"+ row["date"] +".csv", labeled_users)
#     save_data_to_file("Cluster_centers", "cluster_centers"+ row["date"] +".csv", cluster_df)

In [ ]:
cluster_data()

In [ ]:
df = pd.read_csv("All_Labeled_users" + "/labeled_users.csv")[["user_id","label","start_date"]]
len(df.index)

In [ ]:
df.memory_usage()

In [ ]:
def generate_normalized_statistics():
    stats = [np.min, np.mean, np.max, np.std]
    column_names = columns={'mean': 'mean','min': 'amin', 'max': 'amax', 'std':'std'}
    
    #get sample feature names and create aggregats eg {'std_post_frequency': [np.mean, np.std, np.min, np.max]}
    df = pd.read_csv("All_Labeled_users" + "/labeled_users.csv").drop(columns =["user_id", "start_date"])
    features = df.drop(columns =["label"]).columns.values
    aggreagates = { feat : stats for feat in features }
    
#     df = pd.read_csv("Labeled_users" + "/labeled_users" + str(start_date) + ".csv").drop(columns =["user_id"])
        
    stats = (df.groupby(['label']).agg(aggreagates).rename(column_names))
    all_feature = pd.read_csv("Cluster_Data_All" + "/cluster.csv").drop(columns =["user_id", "start_date"])
    divisors = []
    for feat in all_feature.columns.values:
        max_v = all_feature[feat].max()
        divisors.append(max_v)
        divisors.append(max_v)
        divisors.append(max_v)
        divisors.append(max_v)
    normalized = stats.div(divisors, axis='columns')
    return normalized.T
#     stats_trans_df = stats.T
        
#     save_data_to_file_with_index("All_Normalized_ClustersStatistics", "clusters_stats.csv", stats_trans_df)

In [ ]:
norm_stats = generate_normalized_statistics().round(5)
display(HTML(norm_stats.to_html()))

In [ ]:
print(norm_stats.to_latex())

In [ ]:
def generate_normalized_statistics_with_new_feature():
    stats = [np.min, np.mean, np.max, np.std]
    column_names = columns={'mean': 'mean','min': 'amin', 'max': 'amax', 'std':'std'}
    
    #get sample feature names and create aggregats eg {'std_post_frequency': [np.mean, np.std, np.min, np.max]}
    df = pd.read_csv("All_Labeled_users" + "/with_new_feature_labeled_users.csv").drop(columns =["user_id", "start_date"])
    features = df.drop(columns =["label"]).columns.values
    aggreagates = { feat : stats for feat in features }
    
#     df = pd.read_csv("Labeled_users" + "/labeled_users" + str(start_date) + ".csv").drop(columns =["user_id"])
        
    stats = (df.groupby(['label']).agg(aggreagates).rename(column_names))
    all_feature = pd.read_csv("Cluster_Data_All" + "/cluster.csv").drop(columns =["user_id", "start_date"])
    divisors = []
    for feat in all_feature.columns.values:
        max_v = all_feature[feat].max()
        divisors.append(max_v)
        divisors.append(max_v)
        divisors.append(max_v)
        divisors.append(max_v)
    normalized = stats.div(divisors, axis='columns')
    return normalized.T
#     stats_trans_df = stats.T
        
#     save_data_to_file_with_index("All_Normalized_ClustersStatistics", "clusters_stats.csv", stats_trans_df)

In [ ]:
norm_stats = generate_normalized_statistics_with_new_feature().round(5)
display(HTML(norm_stats.to_html()))

In [ ]:
print(norm_stats.to_latex())

In [ ]:
def generate_custer_statistics():
    stats = [np.mean, np.std, np.min, np.max]
    column_names = columns={'mean': 'mean','std': 'stddev', 'amin': 'min', 'amax': 'max'}
    
    #get sample feature names and create aggregats eg {'std_post_frequency': [np.mean, np.std, np.min, np.max]}
    df = pd.read_csv("All_Labeled_users" + "/labeled_users.csv").drop(columns =["user_id", "start_date"])
    features = df.drop(columns =["label"]).columns.values
    aggreagates = { feat : stats for feat in features }
    
#     df = pd.read_csv("Labeled_users" + "/labeled_users" + str(start_date) + ".csv").drop(columns =["user_id"])
        
    stats = (df.groupby(['label']).agg(aggreagates).rename(column_names))
    stats_trans_df = stats.T
        
    save_data_to_file_with_index("All_ClustersStatistics", "clusters_stats.csv", stats_trans_df)


In [ ]:
generate_custer_statistics()

In [ ]:
df = pd.read_csv("All_ClustersStatistics" + "/clusters_stats.csv")

In [ ]:
df = pd.read_csv("All_ClustersStatistics" + "/clusters_stats.csv")
display(HTML(df.reset_index(level=[0]).to_html()))

In [ ]:
print(df.to_latex())

In [ ]:
df2 = df.reset_index(level=[0])

In [ ]:
clust = [str(i) for i in range(0, 7)]
clust

In [ ]:
all_clusters = [str(i) for i in range(0, 7)]
base = ["index", "feature", "stat"]
all_col = base.extend(all_clusters)

In [ ]:
base

In [ ]:
def seperate_cluster_data(df2):
    df2.columns = ["index", "feature", "stat", "0","1","2","3","4","5","6"]
    all_feature = df2['feature'].drop_duplicates()
    all_clusters = df2.columns.values
    print(all_clusters)
    for clust in all_clusters:
        data = []
        for feat in all_feature:
            print(clust)
            stats = df2.loc[df2["feature"] == feat][clust]
            row = [feat]
            row.extend(stats)
            data.append(row)

        stat_df = pd.DataFrame(data, columns = ["feature_name", "mean", "std", "min", "max"])
        save_data_to_file_with_index("ClusterSeparetStats", str(clust) + "_clusters_stats.csv", stat_df)

In [ ]:
seperate_cluster_data(df2)

In [ ]:
cluster_0 = pd.read_csv("ClusterSeparetStats" + "/0_clusters_stats.csv")
display(HTML(cluster_0.to_html()))
#print(cluster_0.to_latex())

In [ ]:
cluster_1 = pd.read_csv("ClusterSeparetStats" + "/1_clusters_stats.csv")
display(HTML(cluster_1.to_html()))
#print(cluster_1.to_latex())

In [ ]:
cluster_2 = pd.read_csv("ClusterSeparetStats" + "/2_clusters_stats.csv")
display(HTML(cluster_2.to_html()))
#print(cluster_2.to_latex())

In [ ]:
cluster_3 = pd.read_csv("ClusterSeparetStats" + "/3_clusters_stats.csv")
display(HTML(cluster_3.to_html()))
#print(cluster_3.to_latex())

In [ ]:
cluster_4 = pd.read_csv("ClusterSeparetStats" + "/4_clusters_stats.csv")
display(HTML(cluster_4.to_html()))
#print(cluster_4.to_latex())

In [ ]:
cluster_5 = pd.read_csv("ClusterSeparetStats" + "/5_clusters_stats.csv")
display(HTML(cluster_5.to_html()))
#print(cluster_5.to_latex())

In [ ]:
cluster_6 = pd.read_csv("ClusterSeparetStats" + "/6_clusters_stats.csv")
display(HTML(cluster_6.to_html()))
#print(cluster_6.to_latex())

In [ ]:
cluster_7 = pd.read_csv("ClusterSeparetStats" + "/7_clusters_stats.csv")
print(cluster_7.to_latex())

In [ ]:
cluster_8 = pd.read_csv("ClusterSeparetStats" + "/8_clusters_stats.csv")
print(cluster_8.to_latex())